<a href="https://colab.research.google.com/github/ManJ-PC/redes-neurais/blob/master/vetoresDePalavras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Palavras Embutidas ou Vetores de Palavras (Word Embeddings)
Adaptado do livro do Chollet (Cap. 6.1: [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff)).

Este código mostra uma outra maneira para representar palavras e continua no mesmo exemplo de análise de sentimentos do código apresentado neste vídeo: [Análise de Sentimento com Keras](https://youtu.be/fG706DT0dOc)


Este vídeo explica o que são e as vantagens de se usar a representação por palavras embutidas (word embeddings): [Word Embeddings, Palavras Embutidas ou Vetores de Palavras](https://youtu.be/Gli2fxRtQlg)

### Rodando este código localmente  

Como este exemplo envolve dois arquivos muito grandes que terão que ser baixados da Internet e descompactados, ao invés de executar este notebook jupyter usando o servidor remoto da google, iremos executá-lo localmente. No lado superior esquerdo da tela do Colab tem um botão que você pode usar para trocar do modo remoto (hosted runtime) para o modo local (local runtime). Se sua máquina já não tiver preparada para rodar o keras e o jupyter tente seguir as inscruções abaixo que eu usei para instalar em uma máquina com Ubuntu 18.04 e GPU Nvidia GTX 1070 (e boa sorte pois você vai precisar !!!):

#### Instalando Tensorflow no Ubuntu 18.04

Eu segui as instruções disponíveis aqui (usando miniconda):
[tensorflow-gpu-installation-made-easy-use-conda-instead-of-pip](https://towardsdatascience.com/tensorflow-gpu-installation-made-easy-use-conda-instead-of-pip-52e5249374bc)

#### Instalando Keras
conda activate tf_gpu

conda install keras

#### Instalando jupyter localmente
Eu segui as instruções disponíveis aqui (use conda ao invés de pip se não quiser ter mais dor de cabeça do que o necessário):
[local-runtimes](https://research.google.com/colaboratory/local-runtimes.html)

#### Instalando o matplotlib
conda install matplotlib




##Aprendendo Vetores de Palavras a partir de Dados



### Carregando o IMDB
A seguir o banco de treinamento e teste é carregado e ajustado para um formato aceito como entrada para a rede neural. Notem que as sentenças vão ser cortadas nas primeiras 20 palavras (é um parâmetro importante neste caso e se perderá informação).





In [ ]:
import keras
keras.__version__

from keras.datasets import imdb
from keras import preprocessing

# Quantidade de palavras do dicionário
max_features = 10000
# Tamanho máximo das sentenças
maxlen = 20

import numpy as np

# Carregando o conjunto de treinamento e teste IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)


# Convertendo para um formato que a rede neural aceita, cortando cada sentença
# para um tamanho fixo = maxlen
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

print(x_train[0])

Using TensorFlow backend.


[  65   16   38 1334   88   12   16  283    5   16 4472  113  103   32
   15   16 5345   19  178   32]


### Criando o modelo e treinando

Novamente uma arquitetura sequencial em camadas é criada, mas agora com uma camada de "Embedding" . O parâmetro 10000 é o total de palavras do seu vocabulário que também é o tamanho da camada de entrada com cada palavra sendo representada usando a codificação uma-quente (one-hot encoding). O parâmetro 8 é o tamanho do vetor para a nova representação que será aprendida. Assista ao vídeo sobre palavras embutidas para entender os conceitos de one-hot encoding e word embedding:  [Word Embeddings, Palavras Embutidas ou Vetores de Palavras](https://youtu.be/Gli2fxRtQlg)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()

# 10000 = total de palavras no dicionário, 8 = tamanho do vetor embutido,
# maxlen = tamanho máximo, em palavras, de cada sentença
model.add(Embedding(10000, 8, input_length=maxlen))

# Achata o vetor 3D em um 2D de dimensão "total de amostras X maxlen*8"
model.add(Flatten())

# A ultima camada é uma completamente conectada (Dense)
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

# Note no código abaixo que diferentemente do exemplo anterior, o conjunto de
# validação e teste é gerado automaticamente através do parâmetro validation_split
# igual a 0.2 (ou seja, 0.8 ou 80% para treinamento e 0.2 ou 20% para validação)
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

W0920 17:43:22.117188 139738181203712 deprecation_wrapper.py:119] From /home/pistori/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0920 17:43:22.136806 139738181203712 deprecation_wrapper.py:119] From /home/pistori/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0920 17:43:22.139297 139738181203712 deprecation_wrapper.py:119] From /home/pistori/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0920 17:43:22.286251 139738181203712 deprecation_wrapper.py:119] From /home/pistori/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Pl

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


W0920 17:43:23.076895 139738181203712 deprecation_wrapper.py:119] From /home/pistori/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 20000 samples, validate on 5000 samples
Epoch 1/10


InternalError: ignored

Veja pela saída do comando model.fit que resultado obtido em acurácia na validação (val_acc)  é pior do que no exemplo anterior, sem word embeddings, mas é porque usamos apenas as primeiras 20 palavras de cada comentários e uma arquitetura mais simples depois da camada de embutimento. Melhorando a arquitetura e ajustando alguns parâmetros dá para melhorar bem. Aqui foi só uma exemplo para entender como usar uma camada de embeddings. Abaixo será criado um outro modelo mas agora usando transfer learning

## Usando Vetores de Palavras (Word Embeddings) pré-treinados (Transfer Learning)

Os códigos a seguir irão utilizar uma representação de vetores de palavras previamente gerada através do algoritmo "GloVe" que foi treinado usando todos os textos em Inglês da Wikipedia em 2014. Uma outra alternativa ao GloVe, também muita usada, é o Word2Vec também disponível no Keras.




### Download dos comentários do IMDB original

Ao invés de usar a função do Keras que facilita carregar o IMDB já codificado (usando o dicionário de números) o exemplos abaixo carregará os comentários em formato "bruto" (raw) como sequência de caracteres. Assim ficará mais fácil reaproveitar os códigos em outros problemas.

IMPORTANTE: Você precisa fazer manualmente o download do IMDB, de 80Mb, no site [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/), descompactar em uma pasta na sua máquina e alterar a inicialização da variável "imdb_dir" abaixo (/home/pistori/Downloads/aclImdb deve ser trocado pela pasta que foi criada na sua máquina)

In [ ]:
import os

imdb_dir = '/home/pistori/Downloads/aclImdb/'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

# Dê uma olhada na estrutura de pastas do banco que você baixou. Os códigos
# abaixo apenas navegam por estas pastas lendo os comentários e colocando na
# lista de comentários (texts). # Também monta a lista "labels" com a
# classificação, positiva ou negativa (1 ou 0) de cada comentário.

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

print(texts[55]) # Mostrando o comentário 55

### Transformar os textos em um vetor de símbolos atômicos (tokens)

A atomização (tokenização) consiste em identificar e separar os elementos do texto que devem ser tratados como unidades básicas (atómos). Neste exemplo, os átomos serão as palavras (mas poderiam, em outra situação, ser caracteres ou mesmo sentenças).

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # Apenas as primeiras 100 palavras do comentário serão usadas
training_samples = 1000  # Serão usadas apenas 1000 palavras para treinamento
validation_samples = 5000  # e 5000 para validação
max_words = 10000  # Tamanho do dicionário de palavras, neste caso, 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Foram encontrados %s símbolos atômicos ou tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Formato do tensor com os comentários:', data.shape)
print('Formato do tensor com as classificações:', labels.shape)

### Aleatorizando e dividindo entre conjunto de treinamento e validação

In [ ]:
# Aleatoriza os dados e depois divide em treinamento e validação

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

### Baixando e processandos as palavras embutidas pré-treinadas com o GloVe

Entre no site [https://nlp.stanford.edu/projects/glove/](https://nlp.stanford.edu/projects/glove/), baixe o arquivo `glove.6B.zip` e descompacte (é um arquivo com cerca de 800Mb). Ajuste no código abaixo a variável "glove_dir" para apontar para a pasta onde você descompactou o arquivo (deve ter um arquivo chamado glove.6B.100d.txt nesta pasta). Será criado uma mapeamento, embeddings_index, onde cada palavra será associada ao seu vetor embutido de 100 dimensões (no arquivo compactado tem também opções para usar vetores de 50, 200, 300 dimensões).

In [ ]:
glove_dir = '/home/pistori/Downloads/glove/'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total de vetores de palavras encontrados na base Glove = %s' % len(embeddings_index))



In [ ]:
print('Vetor embutido correspondente à palavra love = ',embeddings_index['love'])


Abaixo será construída a matriz (tensor) com as palavras embutidas para cada uma das 10.000 do IMDB que iremos usar. Pode acontecer de uma palavras não ter um vetor embutido no GloVe e neste caso, o vetor ficará todo zerado (pode ser, por exemplo, uma palavras com erro de Inglês ou coisas estranhas que as pessoas escrevem nos comentários, como YAAAAHHHH)

In [ ]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Abaixo será construída a rede neural que usaremos para aprender a classificar os comentários (ainda com uma camada de embutimento/embedding 'vazia')

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

### Colocando as palavras embutidas do GloVe no modelo


Os comandos abaixo ajustam os parâmetros da camada de embedding para corresponderem às palavras que baixamos da Internet (GloVe). Note que estamos indicando também que esta camada não deve ser treinada a partir dos dados (trainable=False)

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

### Usando o modelo para aprender e avaliando o resultado

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

Mostrando um gráfico do desempenho entre as épocas

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Acurácia Treino')
plt.plot(epochs, val_acc, 'b', label='Acurácia Val.')
plt.title('Acurácia no treinamento e na validação')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Perda Treino')
plt.plot(epochs, val_loss, 'b', label='Perda Validação')
plt.title('Perda no treinamento e na validação')
plt.legend()

plt.show()

A acurácia novamente é menor do que no exemplo anterior, mas dá para melhorar bastante mexendo, por exemplo, no total de exemplos usados para treinamento (bem menos neste caso do que no exemplo anterior)

Abaixo temos a avaliação do modelo no conjunto independente de teste. Note que é preciso preparar o conjunto de teste também, como fizemos no treinamento.

In [ ]:
test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [ ]:
model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)